In [8]:
import pandas as pd

data_dir = '../data/jp_morgan/sorted'
data_ = 'ds3'

sampling = False
if sampling:
    data_name = f'sample_{data_}'
else:
    data_name = f'final_{data_}_v3'

df = pd.read_parquet(f'{data_dir}/transaction_metrics_{data_name}.parquet')

### Crossborder Flag

In [9]:
def pd_add_crossborder_flag(df):
    party_country_df = df.groupby('party_Id').agg({'party_Country': lambda x: pd.Series.mode(x)[0] if not pd.Series.mode(x).empty else None})
    party_country_df.columns = ['entity_Country']

    df_cols = df.columns.tolist() + ['is_crossborder']

    df= df.merge(party_country_df, how='left', left_on='party_Id', right_index=True)
    df['is_crossborder'] = (~((df['party_Country'] == df['cparty_Country']) | ((df['cparty_Country'].isnull()) & (df['party_Country'] == df['entity_Country'])))).astype(int)

    return df[df_cols]

df = pd_add_crossborder_flag(df)

In [10]:
df.to_parquet(f'{data_dir}/transaction_metrics_{data_name}_cbflag.parquet')